## Week 2: Linear Regression - part I

Instructor: Cornelia Ilin <br>
Email: cilin@ischool.berkeley.edu <br>

#### ``Objectives``

1. Supervised ML: predict continous target variables with regression analysis

2. Gradient Descent for linear regression (stochastic and batch)

3. Use AI/ML to track the progression of diabetes

#### ``Motivation``

1. Start with simple predictive models (if realistic/possible)

2. AI/ML can help predict what types of people are likely to be afflicted with diabetes and how the disease is likely to progress based upon a person’s weight and lifestyle.

#### ``Data``

[Paper](https://tibshirani.su.domains/ftp/lars.pdf): Efron et al. (2004): “Least Angle Regression,” Annals of Statistics (with discussion), 407–499.

[Data description](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html): 10 baseline variables (age, sex, body mass index, average blood pressure, and six blood serum measurements) and the response of interest (a quantitative measure of disease progression one year after baseline readings) for n = 442 diabetes patients.

### Step 1: Import libraries

In [1]:
# standard libraries
import pandas as pd
import numpy as np
import os

# ML/stats
from mlxtend.plotting import heatmap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# visualizations
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

### Step 2: Define functions

In [2]:
def lin_regplot(X, y, y_pred):
    plt.scatter(X, y, c='steelblue', edgecolor='white', s=70)
    plt.plot(X, y_pred, color='black', lw=2)    
    return 

---
### Step 3: Read data

---

In [ ]:
df = pd.read_csv(
    'https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt',
    delimiter='\t'
)

df.head(2)

---
### Step 4: Data preprocessing

---

``rename columns``

In [ ]:
df.columns = [
    'age', 'sex', 'bmi', 'map',
    'tc', 'ldl', 'hdl', 'tch',
    'ltg', 'glu', 'disease_progression']

df.head(2)

``split data into training and test``

In [ ]:
# define outcome #
##################
y = df['disease_progression'].values

# define features #
###################
X = df[['age', 'sex', 'bmi', 'map', 'tc', 'ldl', 'hdl', 'tch', 'ltg', 'glu']].values

# split #
#########
split = (0.7,0.3) #70% training and 30% test
shuffle = np.random.permutation(np.arange(y.shape[0])) # very important to shuffle the data. Why?
X, y = X[shuffle], y[shuffle]

splits = np.multiply(len(y), split).astype(int) 
X_train, X_test = np.split(X, [splits[0]])
y_train, y_test = np.split(y, [splits[0]])


# df for y_train and X_train #
##############################
# easier for EDA later on
df_train = pd.concat(
    [pd.DataFrame(y_train), pd.DataFrame(X_train)],
    axis=1
)

# rename columns
df_train.columns=['disease_progression', 'age', 'sex', 'bmi', 'map',
    'tc', 'ldl', 'hdl', 'tch',
    'ltg', 'glu']

``feature and outcome scalling for optimal performance``

Can be done in two ways:

normalization: rescaling of the features to a range of [0,1]. Also called min-max scaling. Mostly used when we need values in a bounded interval (you can do this manually, but you can also use the MinMaxScaler() method that you can find in sklearn.preprocessing module).

standardization: center the feature columns at mean 0 and std 1. Prefered over normalization, as it preservers useful information about outliers and makes the algorithm less sensitive to them. This is what we will be implementing today.

In [ ]:
# standardize
sc_x = StandardScaler()
sc_y = StandardScaler()

X_train_std = sc_x.fit(X_train).transform(X_train)
X_test_std = sc_x.fit(X_train).transform(X_test)

y_train_std = sc_y.fit(np.expand_dims(y_train, axis=1)).transform(np.expand_dims(y_train, axis=1)).flatten()
y_test_std = sc_y.fit(np.expand_dims(y_train, axis=1)).transform(np.expand_dims(y_test, axis=1)).flatten()

Note how we fit the StandardScaler() only on the training data, and use the mean and sd parameters to transform the test data!

---
### Step 5: Exploratory Data Analysis (EDA)
---

Prior to training a model it is important to perform exploratory data analysis. The idea is to detect any missing data, the presence of outliers, the feature distribution, and the relationship between features and outcome. Note that typically there is a back and forth between Step 4 and 5, depending on how satisfied you are with the results of the EDA.

`` check if any missing values``

In [ ]:
df_train.isna().sum()

``correlation matrix of outcome and feature variables``

In [ ]:
cm = df_train.corr().values
hm = heatmap(
    cm,
    row_names=df_train.columns,
    column_names=df_train.columns,
    figsize=(8, 8)
)

<span style="color:chocolate"> *Question:*</span> Which features seem to be most informative to predict diabetes progression?

``histogram of features and outcome``

Let's focus on the outcome variable (disease_progression) and BMI for the moment.

In [ ]:
columns = ['disease_progression', 'bmi']
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,3))
for idx, ax in enumerate(axes.flatten()):
    # plot histogram
    ax.hist(df_train[columns[idx]])
    # set xlabel
    ax.set_xlabel(columns[idx])

---
### Step 6: Analysis - Linear Regression with one variable
---

We will begin with a single-variable linear regression to predict 'disease_progression' from 'bmi'.

The idea behind linear regression is to obtain the best-fitting straight line through the examples in the training data.

There are two ways you can fit the training data:
 - using a closed-form solution: $(X'X)^{-1}(X'y)$
 - using an iterative solution: gradient descent (stochastic, batch)

Today we will be focusing on **gradient descent** and we will implement this algorithm from scratch. 

In [ ]:
# define features of interest
temp_X_train_std = X_train_std[:,2] # choose only BMI

# exapand dimension so that temp_X_train_std is a 2D array
temp_X_train_std = np.expand_dims(temp_X_train_std, axis=1) 

# define shape of temp_X_train_std
temp_shape_X_train_std = temp_X_train_std.shape[1]

``Linear Regression, w/ Batch Gradient Descent``

In [ ]:
Image(filename='./images/linear.png', width=500)

Let j represent features, j = [1,...,m]

Let i represent observations, i = [1, ..., n]


``Net input function``:

z = $w_{0} \cdot 1 + w_{2} \cdot x_{1} + w_{2} \cdot x_{2} + ... + w_{m} \cdot x_{m}$

``Linear activation function``:

$\phi(z)= w_{0} \cdot 1 + w_{2} \cdot x_{1} + w_{2} \cdot x_{2} + ... + w_{m} \cdot x_{m}$


`` Compute error``:

$(y\_actual_{i} - \phi(z_{i}))$, , where $\phi(z_{i})$ is $y\_pred_{i}$

``Update weights``:

$w_{j} := w_{j} + \Delta w_{j}, \forall j \in {1,...,m}$

$\Delta w_{j} = \eta \sum _{i}(y\_actual_{i} - \phi(z_{i}))\cdot x_{i,j}, \forall j \in {1,...,m}, \forall i \in {1,...,n}$


``Threshold function``:

$\tau(z) = w_{0} \cdot 1 + w_{2} \cdot x_{1} + w_{2} \cdot x_{2} + ... + w_{m} \cdot x_{m}$


`` Note on Batch Gradient Descent``:

The weights are updated using **all training examples** (see the summation operator in the equation $\Delta w_{j}$)

In [ ]:
# Step 1: initalize hyperparameters
eta = 0.001
n_iter = 10
random_state = 1

# Step 2: initalize weights (to small random numbers)
rgen = np.random.RandomState(random_state)
w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + temp_shape_X_train_std)

# Step 3: initialize cost lists
cost_ = []

# Step 4: compute net input, linear activation function, update weights, track cost
def fit(X, y):
    for iter_ in range(n_iter):
        # compute the net input function
        net_input = np.dot(X, w_[1:]) + w_[0]

        # compute the linear activation function; this gives us y_pred
        temp_y_pred = net_input
        
        # compute error
        errors = (y - temp_y_pred)
        #print(y, temp_y_pred)

        # update weights (notice how weights are updated using all training examples!)
        w_[1:] += eta * X.T.dot(errors)
        w_[0] += eta * errors.sum()

        # compute cost (we find weights by minimizing costs; in the Linear model the cost is SSE/2)
        cost = (errors**2).sum()/2.0  
        cost_.append(cost)
        
    return cost_

# Step 5: compute the threshold (linear) function
def predict(X):
    net_input = np.dot(X, w_[1:]) + w_[0]
    
    # threshold function
    y_pred = net_input
    
    return y_pred

Model fit and prediction

In [ ]:
%time cost_ = fit(temp_X_train_std, y_train_std)
y_train_std_pred = predict(temp_X_train_std)

# plot cost
plt.plot((range(1, len(cost_) + 1)), cost_, marker='o');
plt.xlabel('epochs (n_iter)');
plt.ylabel('SSE (cost)'); # number of times y_actual != y_pred

plt.title('Adeline convergence with BGD. Learning rate = ' + str(eta));

Evaluation

In [ ]:
print('MSE:', mean_squared_error(y_train_std, y_train_std_pred))

<span style="color:chocolate"> *Exercise:*</span> 

Try increasing the number of iterations and see what happens.

Then see what happens if you change the learning rate.

If the learning rate is too large (set eta = 0.02): instead of minimizing the cost function, the error become larger after each epoch. This happens because we overshoot the global minimum.

If the learning rate is too small (set eta = 0.0001): the cost decreases but the learning rate is so small that the algo requires lots of epochs to converge (this can be very computational expensive with large datasets!)

In [ ]:
Image(filename='./images/effect_of_eta.png', width=500)

The left subfigure shows that if choose the "right learning rate", the cost decreases gradually, moving in the direction of the global minimum.

The right subfigure shows what happens if we choose a learning rate that is too large - we miss the global minimum.

``Linear Regression, w/ Stochastic Gradient Descent``

<span style="color:chocolate"> *Update weights:*</span>

$w_{j} := w_{j} + \Delta w_{j}, \forall j \in {1,...,m}$

$\Delta w_{j} = \eta (y\_actual_{i} - \phi(z_{i}))\cdot x_{i,j}, \forall j \in {1,...,m}, \forall i \in {1,...,n}$

`` Note on Stochastic Gradient Descent``:

The weights are updated using **one training example at a time** (no summation operator in the equation $\Delta w_{j}$)


As a result, the SGD typically reaches convergence much faster than BGD because of the more frequent weight updates.

In [ ]:
# Step 1: initalize hyperparameters
eta = 0.001
n_iter = 10
random_state = 1

# Step 2: initalize weights (to small random numbers)
rgen = np.random.RandomState(random_state)
w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + temp_shape_X_train_std)

# Step 3: initialize cost lists
cost_ = []


# Step 4: compute net input, linear activation function, update weights, track cost
def fit(X, y):
    for iter_ in range(n_iter):
        cost = []
        # shuffle the data after each iteration to avoid repetitive cycles when we optimize the cost function
        shuffle = np.random.permutation(np.arange(X.shape[0]))
        X, y = X[shuffle], y[shuffle]
        
        
        # compute the net input function
        net_input = np.dot(X, w_[1:]) + w_[0]

        # compute the linear activation function; this gives us y_pred
        temp_y_pred = net_input
        
        # iterate through all i in X,y
        for idx, (xi, y_actual) in enumerate(zip(X, y)):
            # compute error
            errors = (y_actual - temp_y_pred[idx])
            

            # update weights (notice how weights are updated using one training example at a time!)
            w_[1:] += eta * xi.dot(errors)
            w_[0] += eta * errors

            # compute cost (we find weights by minimizing costs; in the Linear model the cost is SSE/2)
            cost.append(errors**2/2.0)
            
        avg_cost = sum(cost)/len(y)
        cost_.append(avg_cost)
    return cost_

# Step 5: compute the threshold (linear) function; it's just the net input function in the linear model
def predict(X):
    net_input = np.dot(X, w_[1:]) + w_[0]
    # threshold function
    y_pred = net_input
    
    return y_pred

Model fit and prediction

In [ ]:
%time cost_ = fit(temp_X_train_std, y_train_std)
y_train_std_pred = predict(temp_X_train_std)

# plot cost
plt.plot((range(1, len(cost_) + 1)), cost_, marker='o');
plt.xlabel('epochs (n_iter)');
plt.ylabel('SSE (cost)'); # number of times y_actual != y_pred

plt.title('Adeline convergence with BGD. Learning rate = ' + str(eta));

Evaluation

In [ ]:
print('MSE:', mean_squared_error(y_train_std, y_train_std_pred))

``The regression line``

In [ ]:
lin_regplot(temp_X_train_std, y_train_std, y_train_std_pred)
plt.xlabel(' Body Mass Index (standardized)')
plt.ylabel('Disease progression (standardized)')
plt.show()

As you can see, the linear regression line reflects the general trend that diabetes progression tends to increase with the BMI. This observation makes sense but the data also tells us that BMI does not explain disease progression very well in many cases.

---
TO DOs in the breakout rooms:

   - [a] What is the MSE on the test data?  <br>
   - [b] How would you improve the MSE of the model? Implement your idea.

---